In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('Diff-stock.csv')

Here I am impporting the variables which currently I am working on. Here I am trying to find out that for example, this is a portfolio of Govt which includes share market, Fed Balance Sheet, Federal Funds rate and the Loss Function and I amhere trying to find out that on which variable Govt. should give more focus as well as more weight in the time of taking decision.

Here, the datasets are twice differenced to ensure stationarity.

In [4]:
df

,DATE,S&P 500,TA,CC,FFR,LF
0,2010-03,0.029018,0.008611,-0.000738,0.021659,-1.133584
1,2010-04,-0.042481,-0.000267,-0.003030,-0.004270,1.019559
2,2010-05,-0.100183,-0.009751,-0.000112,-0.031505,3.585353
3,2010-06,0.030143,-0.003164,0.001554,-0.026290,-1.340778
4,2010-07,0.121904,-0.002388,-0.000532,0.029892,-2.219644
...,...,...,...,...,...,...
113,2019-08,-0.031300,0.001927,-0.003976,-0.302333,0.109370
114,2019-09,0.035198,0.017413,0.006447,0.194290,0.050563
115,2019-10,0.003379,0.032283,-0.003852,-0.130516,-0.187102
116,2019-11,0.013160,-0.023511,0.005203,-0.063022,-0.018272


In [5]:
df = df.set_index('DATE')
df

,S&P 500,TA,CC,FFR,LF
DATE,,,,,
2010-03,0.029018,0.008611,-0.000738,0.021659,-1.133584
2010-04,-0.042481,-0.000267,-0.003030,-0.004270,1.019559
2010-05,-0.100183,-0.009751,-0.000112,-0.031505,3.585353
2010-06,0.030143,-0.003164,0.001554,-0.026290,-1.340778
2010-07,0.121904,-0.002388,-0.000532,0.029892,-2.219644
...,...,...,...,...,...
2019-08,-0.031300,0.001927,-0.003976,-0.302333,0.109370
2019-09,0.035198,0.017413,0.006447,0.194290,0.050563
2019-10,0.003379,0.032283,-0.003852,-0.130516,-0.187102


The below I am going to calculate the mean of the variables:

In [6]:
df.mean()

S&P 500    6.270932e-07
TA         1.111607e-04
CC        -1.379178e-05
FFR       -1.592726e-04
LF         4.267676e-04
dtype: float64

In [21]:
df.mean().mean()

7.309820338982404e-05

In [22]:
returns = df

Now, I will calculate the weight of the variable with the below automated code:

In [23]:
# the objective function is to minimize the portfolio risk
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(returns.cov()).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.
        # The expected daily return of our portfolio and we want to be at greater than 0.002352
        {"type": "ineq", "fun": lambda x: np.sum(returns.mean()*x)-0.00008})
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(returns.shape[1]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./returns.shape[1] for x in range(returns.shape[1])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
optimized_results

     fun: 6.857209166780858e-05
     jac: array([3.76944849e-04, 1.03787687e-04, 7.98662586e-06, 5.89357576e-04,
       3.01438127e-03])
 message: 'Optimization terminated successfully'
    nfev: 48
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([4.81545060e-02, 8.15308785e-01, 7.92873439e-02, 5.72493651e-02,
       3.46944695e-18])

The optimum weights are the array x and we can retrieve them as follows:

In [24]:
optimized_results.x

array([4.81545060e-02, 8.15308785e-01, 7.92873439e-02, 5.72493651e-02,
       3.46944695e-18])

We can check that the weights sum up to 1:

In [25]:
# we get 1
np.sum(optimized_results.x)

1.0

We can see that the expected return of the portfolio is:

In [26]:
np.sum(returns.mean()*optimized_results.x)

8.044870819705533e-05

Which is almost 0.00008044 (some rounding errors) which was our requirement.

In [27]:
symbols = ['S&P 500', 'TA', 'CC', 'FFR', 'LF']

In [28]:
pd.DataFrame(list(zip(symbols, optimized_results.x)), 
                       columns=['Symbol', 'Weight'])

,Symbol,Weight
0,S&P 500,4.815451e-02
1,TA,8.153088e-01
2,CC,7.928734e-02
3,FFR,5.724937e-02
4,LF,3.469447e-18


In [ ]:
0.04815451, 0.8153088, 0.07928734, 0.05724937